# Deep Hedging pipeline

## Evironment setup

Setup the general environment

In [1]:
import os, sys
from pathlib import Path

main_folder = str(Path.cwd().parent)
sys.path.append(main_folder)

import warnings
warnings.filterwarnings("ignore")

## Simulation of NIG random values

Simulation of standard NIG random variables for the JIVR model

In [2]:
from src.utils import *
from src.features.nig_simulation import *

config_file = load_config(os.path.join(main_folder,'cfgs','config_simulation.yml'))
nig_simulation(config_file)

--Generating grid for NIG interpolation--
--NIG simulation starts--
--Progress of NIG simulation: 100.00%
--Simulation completed - NIG values stored in ../data/interim/--


## JIVR Simulation

Simulation of JIVR features

**Remark**: This segment of the pipeline assumes that the NIG random variables have already been simulated.

In [3]:
from src.utils import *
from src.features.jivr_simulation import *

config_file = load_config(os.path.join(main_folder,'cfgs','config_simulation.yml'))
jivr_model = implied_volatily_surface_vec(config_file)
jivr_model.jivr_simulation()

-- JIVR simulation starts --
--Progress of JIVR simulation: 100.00%
--Simulation of JIVR features completed--
-- Simulation of stock price --
-- Simulation completed - JIVR features stored in ../data/processed/--


## Deep Hedging 

Deep RL agent training and hedging strategy generation

**Remark 1**: This segment of the pipeline assumes that the JIVR features have already been simulated.

**Remark 2**: This training example illustrates the performance of the RL agent optimized using the MSE loss, relying solely on the underlying asset as a hedging instrument in the presence of transaction costs with $\kappa_{1}=0.05\%$. The corresponding results are reported in Table 4 of the paper.


In [4]:
from src.utils import *
from src.models.deep_rl_training import rl_agent

config_file = load_config(os.path.join(main_folder,'cfgs','config_agent.yml'))
config_file_agent = config_file["agent"]

config_file = load_config(os.path.join(main_folder,'cfgs','config_simulation.yml'))
config_file_simulation = config_file["simulation"]

hedging_strategy = rl_agent(config_file_simulation,config_file_agent)

-------- Portfolio simulation --------
-------- Simulation completed --------
-------------------------------------------------------------
Deep_agent_RNNFNN_noauto_dropout_50_MSE_TC_0.05_2.0_learned_reebalance_integral_HO_C63ATMpos1P63ATMpos1_softcons_1
-------------------------------------------------------------
-----------------------Training start------------------------
Epoch 1, Time elapsed: 0:03:13.993735  ---- Model saved
  Train - MSE: 23.810, Auto: 0.000, Soft_cons: 0.099
  Valid - MSE: 15.721, Auto: 0.000, Soft_cons: 0.069
  Condition 1: 0.0000000000
Epoch 2, Time elapsed: 0:06:45.861728  ---- Model saved
  Train - MSE: 6.443, Auto: 0.000, Soft_cons: 0.019
  Valid - MSE: 3.502, Auto: 0.000, Soft_cons: 0.005
  Condition 1: 0.0000000000
Epoch 3, Time elapsed: 0:09:33.496113  ---- Model saved
  Train - MSE: 3.040, Auto: 0.000, Soft_cons: 0.005
  Valid - MSE: 2.872, Auto: 0.000, Soft_cons: 0.005
  Condition 1: 0.0000000000
Epoch 4, Time elapsed: 0:12:21.075925  ---- Model saved

INFO:tensorflow:Restoring parameters from Deep_agent_RNNFNN_noauto_dropout_50_MSE_TC_0.05_2.0_learned_reebalance_integral_HO_C63ATMpos1P63ATMpos1_softcons_1.ckpt


-------------------------------------------------------------
Deep_agent_RNNFNN_noauto_dropout_50_MSE_TC_0.05_2.0_learned_reebalance_integral_HO_C63ATMpos1P63ATMpos1_softcons_1
-------------------------------------------------------------
---Inference start---
---Inference end---
--- Hedging startegy stored in ../results/Training/.. ---
